In [ ]:
import os
import numpy as np
import mne
import osl
import yaml
from scipy.io import savemat
import matplotlib.pyplot as plt

In [ ]:
dataset_path = "/gpfs2/well/woolrich/projects/disp_csaky/CTF/rich/16253003_reading_single_trial.ds"
raw = mne.io.read_raw_ctf(dataset_path, preload=True)

In [ ]:
ev = mne.events_from_annotations(raw)

In [ ]:
ev[0][:100]

In [ ]:
raw = raw.crop(tmin=100, tmax=120)

In [ ]:
%matplotlib widget
raw.plot(duration=20, n_channels=50)
e=0

In [ ]:
# print channel types
print(raw.get_channel_types())

In [ ]:
raw.ch_names

In [ ]:
config_text = """
meta:
    event_codes:
        words/hungry: 2
        words/tired: 3
        words/thirsty: 4
        words/toilet: 5
        words/pain: 6
preproc:
  - filter:         {l_freq: 1, h_freq: 25, method: 'iir', iir_params: {order: 5, ftype: butter}}
  - bad_channels:   {picks: 'grad'}
  - bad_segments:   {segment_len: 800, picks: 'grad'}
  - ica_raw:        {picks: 'meg', n_components: 64}
  - ica_autoreject: {picks: 'meg', ecgmethod: 'correlation', measure: 'correlation', threshold: 0.5}
  - find_events:    {min_duration: 0.002}
"""

In [ ]:
raw.ch_names

In [ ]:
raw.set_channel_types({'UADC007-4123': 'eog',
                       'UADC008-4123': 'eog',
                       'UADC009-4123': 'ecg',})

In [ ]:
osl_outdir = "/gpfs2/well/woolrich/projects/disp_csaky/CTF/rich/preproc25hz/oslpy"
config = yaml.load(config_text, Loader=yaml.FullLoader)
dataset = osl.preprocessing.run_proc_chain(config, raw, outdir=osl_outdir, overwrite=True, gen_report=True)

In [ ]:
raw = raw.pick(['mag'])

In [ ]:
raw._raw_times

In [ ]:
raw.info['time']

In [16]:
raw = raw.resample(100)

In [17]:
events = mne.events_from_annotations(raw)[0]

# only keep events with id below 7
events = events[events[:, -1] < 7]
print(events[:100])

Used Annotations descriptions: ['0', '1', '2', '3', '4', '5', '6', 'Btn1', 'Btn2', 'Btn3', 'Btn4', 'Tr0_1', 'Tr0_2', 'Tr0_3', 'Tr0_4', 'Tr0_5', 'Tr0_6', 'Tr0_7', 'Tr1_1', 'Tr1_2', 'Tr1_3', 'Tr1_4']
[[ 3609     0     5]
 [ 3703     0     1]
 [ 3806     0     3]
 [ 3893     0     1]
 [ 3983     0     6]
 [ 4065     0     1]
 [ 4163     0     2]
 [ 4261     0     1]
 [ 4356     0     4]
 [ 4446     0     1]
 [ 4551     0     2]
 [ 4635     0     1]
 [ 4724     0     5]
 [ 4816     0     1]
 [ 4913     0     4]
 [ 5006     0     1]
 [ 5113     0     6]
 [ 5193     0     1]
 [ 5285     0     3]
 [ 5372     0     1]
 [ 5815     0     4]
 [ 5895     0     1]
 [ 5993     0     3]
 [ 6075     0     1]
 [ 6180     0     2]
 [ 6278     0     1]
 [ 6368     0     5]
 [ 6453     0     1]
 [ 6550     0     6]
 [ 6648     0     1]
 [ 6746     0     2]
 [ 6845     0     1]
 [ 6948     0     3]
 [ 7028     0     1]
 [ 7133     0     4]
 [ 7217     0     1]
 [ 7310     0     6]
 [ 7390     0     1]
 [ 7

In [24]:
events[623:633]

array([[76898,     0,     1],
       [77000,     0,     6],
       [77000,     0,     6],
       [77085,     0,     1],
       [77176,     0,     4],
       [77275,     0,     1],
       [77378,     0,     3],
       [77478,     0,     1],
       [77568,     0,     4],
       [77660,     0,     1]])

In [18]:
times = [e[0] for e in events]

In [21]:
shifted = np.array(times[:-1]) - np.array(times[1:])

In [23]:
np.where(shifted == 0)

(array([624, 853]),)